<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import precision_recall_curve, auc, roc_curve, roc_auc_score
import random as r
import matplotlib.pyplot as plt

In [4]:
data = pd.read_parquet('e:\\ds\\hackaton\\joined_mod.parquet')

In [2]:
test = pd.read_parquet('e:\\ds\\hackaton\\test_pairs.parquet')
test.head(1)

,variantid1,variantid2,name_1,categories_1,color_parsed_1,pic_embeddings_resnet_v1_1,main_pic_embeddings_resnet_v1_1,name_bert_64_1,characteristic_attributes_mapping_1,name_2,categories_2,color_parsed_2,pic_embeddings_resnet_v1_2,main_pic_embeddings_resnet_v1_2,name_bert_64_2,characteristic_attributes_mapping_2
0,52076340,290590137,Батарейка AAA щелочная Perfeo LR03/10BL Super ...,Батарейки и аккумуляторы,None,"[0.15417035, 0.41160947, 0.2213532, -0.0197311...","[0.04763528, -0.20136409, 0.29605597, 0.264536...","[-0.28437558, 0.60909724, 0.5972025, -0.523296...","{""Напряжение, В"":[""1.5""],""Бренд"":[""Perfeo""],""Т...",Батарейка AAA щелочная Perfeo LR03/2BL mini Su...,Батарейки и аккумуляторы,None,"[-0.025554053, 0.012488857, 0.43989864, -0.103...","[0.06223978, -0.16145544, 0.26409012, 0.242712...","[-0.3380968, 0.6156224, 0.6428071, -0.57499236...","{""Форм-фактор батареи"":[""AAA""],""Химический тип..."


In [10]:
test_txt = test[['name_1', 'name_2']]

In [5]:
data_1 = data[['name', 'categories', 'variantid1', 'variantid2', 'name_2', 'target']]

In [5]:
data_txt = data[['name', 'name_2']]

---

In [3]:
d_ptrn = re.compile('[0-9]')
c_ptrn = re.compile('[a-zа-яйёё]')
s_ptrn = re.compile('[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]')
def templater(txt):
    digits = re.findall(d_ptrn, txt)
    chars = re.findall(c_ptrn, txt)
    symb = re.findall(s_ptrn, txt)
    digits_pos = list(set([pos for pos, char in enumerate(txt) for c in digits if char == c]))
    chars_pos = list(set([pos for pos, char in enumerate(txt) for c in chars if char == c]))
    symb_pos = list(set([pos for pos, char in enumerate(txt) for c in symb if char == c]))
    
    template = []
    if(len(symb_pos) > 0):
        s = symb_pos
    else:
        s = [0]
    if(len(digits_pos) > 0):
        d = digits_pos
    else:
        d = [0]
    if(len(chars_pos) > 0):
        c = chars_pos
    else:
        c = [0]
        
    for i in range(len(txt)+1):
        if i in symb_pos:
            template.append('#')
        elif i in digits_pos:
            template.append('0')
        elif i in chars_pos:
            template.append('X')
    return ''.join(template)

In [4]:
reg_pattern = re.compile('^[a-zа-яйёё,."-/]+$|[/]+|[,]+$')
def reg(txt):
    return ' '.join([re.sub(reg_pattern, '',t.lower()) for t in re.sub(r'[/]+', r' ', re.sub(r'[()]+', r' ', txt)).split(' ') if len(re.sub(reg_pattern, '',t.lower())) > 0])

In [5]:
def dl_compare(l1, l2): 
    pat = re.compile('[a-zа-яйёё]+$')
    adl = [re.findall(pat, x)[0] for x in l1]
    bdl = [re.findall(pat, x)[0] for x in l2]
    return set(adl) & set(bdl)

In [6]:
def dupl_searcher(pair_df):
    sim = 0
    unlike = 0
    wopair = 0
    for col in pair_df.columns.to_list():   
        a = pair_df[col][0]
        b = pair_df[col][1]
        c = set(a) & set(b)
        sim = sim + len(c)
        for each in c:
            a_times = [idx for idx, val in enumerate(a) if val == each]
            b_times = [idx for idx, val in enumerate(b) if val == each]
            times = np.minimum(len(a_times), len(b_times))
            for t in range(times):
                a.remove(each)
                b.remove(each)
            
        unlike = unlike + np.minimum(len(a), len(b))
        for i in range(np.minimum(len(a), len(b))):
            del a[0]
            del b[0]

        pair_df[col][0] = a
        pair_df[col][1] = b
    wopair = pair_df.applymap(lambda x: len(x)).to_numpy().sum()

    return sim, unlike, wopair

In [7]:
digits_pat = re.compile('^[\d+]+$')
let_pat = re.compile('^[a-zа-яйёё]+$')
dl_pat = re.compile('^[0-9]+[a-zа-яйёё]+$')
every_pat = re.compile('[+\w\s\d-]+')

def pair_analysis(val):
    txt_1 = val[0]
    txt_2 = val[1]
    t1 = reg(unicodedata.normalize("NFKD", txt_1).lower()).split(' ')
    t2 = reg(unicodedata.normalize("NFKD", txt_2).lower()).split(' ')
#-----templates
    token_types = []
    for each in t1+t2:
        if len(each) == 0:
            continue
        token_types.append(templater(each))
    token_types = list(set(token_types))
#===============
    t1_dict = {}
    t2_dict = {}
    for c in token_types:
        t1_dict[c] = [x for x in t1 if templater(x) == c]
        t2_dict[c] = [x for x in t2 if templater(x) == c]
    txt_pair_df = pd.DataFrame(data = (t1_dict, t2_dict))
    return dupl_searcher(txt_pair_df)

In [11]:
%%time
results = test_txt.loc[:].apply(pair_analysis, axis = 1)

CPU times: total: 1min
Wall time: 1min 1s


In [13]:
results_df = pd.DataFrame.from_records(results, columns=['similar', 'opposite', 'single'])
results_df

,similar,opposite,single
0,1,0,5.0
1,1,2,0.0
2,1,0,4.0
3,2,0,4.0
4,2,1,1.0
...,...,...,...
18079,3,2,10.0
18080,0,0,2.0
18081,1,4,4.0
18082,1,0,0.0


In [15]:
# results_df.to_csv('e:\\ds\\hackaton\\results_df.csv', index=False)

In [15]:
results_df.to_csv('e:\\ds\\hackaton\\test_results_df.csv', index=False)

---

---